In [78]:
import os
os.chdir("/home/composersyf/Documents/RelishMBA/MixPanel_Data")

In [79]:
import numpy as np
import pandas as pd
from collections import Counter

In [80]:
import json
with open("All_People_Mixpanel_07_26.json") as people_data:
    people_data=people_data.readlines()[0]
people=json.loads(people_data) #people is stored as a list

In [81]:
import json
with open("All_Events_Mixpanel_07_26.json") as events_data:
    events_data=events_data.readlines()[0]
events=json.loads(events_data) #events is stored as a list

In [82]:
len(people)

2251

In [83]:
people_roles=[]
for p in people:
    try:
        people_roles.append(p['properties']['Role'])
    except KeyError:
        people_roles.append("Unknown")
set(people_roles)

{'Unknown', 'student'}

In [84]:
people_roles=np.array(people_roles)
Counter(people_roles)

Counter({'Unknown': 107, 'student': 2144})

In [85]:
Schools=[]
for p in people:
    try:
        if p['properties']['Role']=="student":
            try:
                Schools.append(p['properties']['School'])
            except KeyError:
                pass
    except KeyError:
        pass
Schools=np.unique(np.array(Schools))
Schools

array(['ASU (W.P. Carey)', 'Alabama (Manderson)', 'American (Kogod)',
       'BYU (Marriott)', 'Berkeley (Haas)', 'Boston College (Carroll)',
       'Boston University (Questrom)', 'CEIBS (CEIBS)',
       'Cambridge (Judge)', 'Carnegie Mellon (Tepper)', 'Chicago (Booth)',
       'Claremont (Drucker)', 'Colorado - Boulder (Leeds)',
       'Columbia (CBS)', 'Cornell (Johnson)', 'Dartmouth (Tuck)',
       'Duke (Fuqua)', 'ESADE Business School (ESADE)', 'Emory (Goizueta)',
       'Forte Network (Forte)', 'George Washington (GW)',
       'Georgetown (McDonough)', 'Georgia (Terry)',
       'Georgia Tech (Scheller)', 'HEC-Paris (HEC)', 'Harvard (HBS)',
       'IE Business School (IE)', 'IESE Business School',
       'IESE Business School (IESE)', 'INSEAD (INSEAD)',
       'Imperial College Business School (ICBS)', 'Indiana (Kelley)',
       'Johns Hopkins (Carey)', 'London Business School (LBS)',
       'MIT (Sloan)', 'Manchester (Alliance)', 'Marquette (GSM)',
       'Maryland (Smith)', 'Mi

In [86]:
Inactive_Schools=['UC Irvine (Merage)','Alabama (Manderson)','University of Illinois (Urbana-Champaign)',
                  'Colorado - Boulder (Leeds)','Marquette (GSM)','Toronto (Rotman)','BYU (Marriott)',
                  'Georgia (Terry)','Pepperdine (Graziadio)','SMU (Cox)','USC (Marshall)','Pittsburgh (Katz)',
                  'Ohio State (Fisher)',"Queen's School of Business (QSB)",'Rochester (Simon)',
                  'George Washington (GW)','Minnesota (Carlson)','Rice (Jones)','Georgia Tech (Scheller)']                  

In [87]:
people_list_by_school_2={}
for s in Schools:
    people_list_by_school_2[s]=[]
for p in people:
    try:
        people_list_by_school_2[p['properties']['School']].append(p['distinct_id'])
    except KeyError:
        pass
total_students=0
for s in people_list_by_school_2:
    total_students+=len(people_list_by_school_2[s])
total_students

2144

In [88]:
people_list_by_school={}
for s in Inactive_Schools:
    people_list_by_school[s]=[]
for p in people:
    try:
        if p['properties']['School'] in Inactive_Schools:
            people_list_by_school[p['properties']['School']].append(p['distinct_id'])
    except KeyError:
        pass
people_list_by_school

{'Alabama (Manderson)': ['4878'],
 'BYU (Marriott)': ['4861', '4816', '4547'],
 'Colorado - Boulder (Leeds)': ['4923'],
 'George Washington (GW)': ['4295',
  '4351',
  '4766',
  '4239',
  '3664',
  '4700',
  '482'],
 'Georgia (Terry)': ['4703'],
 'Georgia Tech (Scheller)': ['3538'],
 'Marquette (GSM)': ['3461'],
 'Minnesota (Carlson)': ['3237', '3031', '2435', '3235', '4232'],
 'Ohio State (Fisher)': ['3343'],
 'Pepperdine (Graziadio)': ['4318', '4957'],
 'Pittsburgh (Katz)': ['4506'],
 "Queen's School of Business (QSB)": ['2393'],
 'Rice (Jones)': ['3250', '2970', '3379', '4492', '3242'],
 'Rochester (Simon)': ['4738', '4317'],
 'SMU (Cox)': ['3310', '3877', '4346', '4655'],
 'Toronto (Rotman)': ['4079', '4278'],
 'UC Irvine (Merage)': ['4741'],
 'USC (Marshall)': ['3210', '4640', '4863', '1565'],
 'University of Illinois (Urbana-Champaign)': ['4742']}

In [ ]:
people_list={}
for s in people_list_by_school:
    for p_id in people_list_by_school[s]:
        people_list[(s,p_id)]={}
        #First / Last name, email address
        for p in people:
            if p['distinct_id']==p_id:
                people_list[(s,p_id)]["first_name"]=p['properties']['First Name']
                people_list[(s,p_id)]["last_name"]=p['properties']['Last Name']
                people_list[(s,p_id)]["email"]=p['properties']['Email']
                people_list[(s,p_id)]['signup_date']=p['properties']['SignUp Date']
                break
        #User Activity: Student Applied Following Filter, view Profile, Company Save, Company Follow, Student visit Search (6 metrics)
        for e in events:
            if (e['name'] in ['Student Applied Following Filter', 'view Profile', 'Company Save', 'Company Follow', 'Student SignIn', 'Student visit SEARCH', 'Student visit Search']) and e['distinct_id']==p_id:
                try:
                    people_list[(s,p_id)][e['name'].lower()]+=1
                except KeyError:
                    people_list[(s,p_id)][e['name'].lower()]=1
people_list

In [90]:
people_df=pd.DataFrame.from_dict(people_list).T
people_df=people_df.fillna(0)
people_df.index.names=["School","user_id"]
people_df.index.names
people_df=people_df.groupby(level="School",axis=0).apply(pd.DataFrame.sort_values,by="signup_date",ascending=False)
people_df=people_df.reset_index(0).iloc[:,1:]
people_df

company follow  \
School                                    user_id                   
Alabama (Manderson)                       4878                  0   
BYU (Marriott)                            4861                  0   
                                          4816                  0   
                                          4547                  0   
Colorado - Boulder (Leeds)                4923                  0   
George Washington (GW)                    4766                  2   
                                          4700                  0   
                                          4351                  0   
                                          4295                  0   
                                          4239                  0   
                                          3664                  0   
                                          482                   5   
Georgia (Terry)                           4703                 10   
Georgia Tech (Scheller)                   3538                  1   
Marquette (GSM)                           3461                  1   
Minnesota (Carlson)                       4232                  0   
                                          3237                  0   
                                          3235                  2   
                                          3031                  0   
                                          2435                  0   
Ohio State (Fisher)                       3343                  2   
Pepperdine (Graziadio)                    4957                  0   
                                          4318                 13   
Pittsburgh (Katz)                         4506                  0   
Queen's School of Business (QSB)          2393                  0   
Rice (Jones)                              4492                  3   
                                          3379                  7   
                                          3250                  0   
                                          3242                  0   
                                          2970                  0   
Rochester (Simon)                         4738                  2   
                                          4317                  0   
SMU (Cox)                                 4655                  0   
                                          4346                  0   
                                          3877                  0   
                                          3310                  0   
Toronto (Rotman)                          4278                  0   
                                          4079                  0   
UC Irvine (Merage)                        4741                  0   
USC (Marshall)                            4863                 14   
                                          4640                  3   
                                          3210                  0   
                                          1565                  0   
University of Illinois (Urbana-Champaign) 4742                  0   

                                                   company save  \
School                                    user_id                 
Alabama (Manderson)                       4878                0   
BYU (Marriott)                            4861                0   
                                          4816                0   
                                          4547                0   
Colorado - Boulder (Leeds)                4923                0   
George Washington (GW)                    4766                2   
                                          4700                0   
                                          4351                0   
                                          4295                0   
                                          4239                0   
                                          3664                2   
     

In [91]:
people_df=people_df.iloc[:,[2,3,4,5,8,6,9,0,1,7]]
people_df=people_df.rename(columns={"student visit search":"visit_search","student applied following filter":"apply_filter",
                                    "view profile":"view_profile","company follow":"follow","company save":"save",
                                    "student signin":"signin"})
people_df

email  \
School                                    user_id                                            
Alabama (Manderson)                       4878                     khouser@underarmour.com   
BYU (Marriott)                            4861                        aeheflin93@gmail.com   
                                          4816                    tadbrinkerhoff@gmail.com   
                                          4547                      ericleon2008@gmail.com   
Colorado - Boulder (Leeds)                4923                      johnrhelmers@yahoo.com   
George Washington (GW)                    4766                      cliff.gillam@gmail.com   
                                          4700                           rschopler@gwu.edu   
                                          4351                          nataliemay@gwu.edu   
                                          4295                    ian.p.laughlin@gmail.com   
                                          4239                    yasin.sebaggala@live.com   
                                          3664                   jordan.schulman@gmail.com   
                                          482                            hillk90@gmail.com   
Georgia (Terry)                           4703                       gussstephen@gmail.com   
Georgia Tech (Scheller)                   3538                         kparbadia@gmail.com   
Marquette (GSM)                           3461            nicholas.winninger@marquette.edu   
Minnesota (Carlson)                       4232                       dbaxter1016@gmail.com   
                                          3237                            buckn035@umn.edu   
                                          3235                           skuplic@umich.edu   
                                          3031                            bruce080@umn.edu   
                                          2435                     ryan.t.alberg@gmail.com   
Ohio State (Fisher)                       3343                     hershgupta911@gmail.com   
Pepperdine (Graziadio)                    4957                             rks5@icloud.com   
                                          4318                        joeharkins@gmail.com   
Pittsburgh (Katz)                         4506                         jophameen@gmail.com   
Queen's School of Business (QSB)          2393               sunandan.sridharan@queensu.ca   
Rice (Jones)                              4492                      tuanquang269@gmail.com   
                                          3379                         vincezito@gmail.com   
                                          3250                       mark.cochran@rice.edu   
                                          3242                              dp@purdytx.com   
                                          2970                               yda1@rice.edu   
Rochester (Simon)                         4738                    jason.a.beaton@gmail.com   
                                          4317                      nehasharma16@gmail.com   
SMU (Cox)                                 4655                              mnoula@smu.edu   
                                          4346                               mmaia@smu.edu   
                                          3877                       jenrdelaney@gmail.com   
                                          3310                             ckoenig@smu.edu   
Toronto (Rotman)                          4278                    mwade.taylor@utoronto.ca   
                                          4079                      vincelauro10@gmail.com   
UC Irvine (Merage)                        4741                          ayush.rc@gmail.com   
USC (Marshall)                            4863                       brownsteint@gmail.com   
                                          4640                     missmelissang@gmail.com   
                                          3

In [92]:
people_df.to_csv("targets_users.csv")

In [16]:
user_4878_events=[]
for e in events:
    if e['distinct_id']=='4878':
        user_4878_events.append(e)
user_4878_events

[{'distinct_id': '4878',
  'name': 'Student Build Profile',
  'properties': {'$lib_version': '2.2.0', 'mp_lib': 'ruby'},
  'sampling_factor': 1,
  'time': 1468501035000},
 {'distinct_id': '4878',
  'name': 'Signup Confirmed as #{self.role}',
  'properties': {'$lib_version': '2.2.0',
   'mp_lib': 'ruby',
   'school': 'Alabama (Manderson)',
   'signup_method': 'LinkedIn'},
  'sampling_factor': 1,
  'time': 1468501061000},
 {'distinct_id': '4878',
  'name': 'Student SignIn',
  'properties': {'$lib_version': '2.2.0', 'mp_lib': 'ruby'},
  'sampling_factor': 1,
  'time': 1468501139000},
 {'distinct_id': '4878',
  'name': 'Student visit Search',
  'properties': {'$browser': 'Internet Explorer',
   '$browser_version': 11,
   '$city': 'Baltimore',
   '$current_url': 'http://www.relishmba.com/students/kristie-matthai/edit',
   '$initial_referrer': '$direct',
   '$initial_referring_domain': '$direct',
   '$lib_version': '2.8.1',
   '$os': 'Windows',
   '$region': 'Maryland',
   '$screen_height': 

In [27]:
for p in people:
    if p['distinct_id']=="4878":
        print(p)

{'properties': {'Recruiting Status': None, 'Expected Grad Year': None, '$timezone': 'America/New_York', 'Email': 'khouser@underarmour.com', 'Function Experience': None, 'Industry Experience': None, '$region': 'Virginia', 'company_visited_count': 5, '$country_code': 'US', 'signup_method': 'LinkedIn', 'SignUp Date': '2016-07-14T13:00:54.000Z', 'School': 'Alabama (Manderson)', 'Role': 'student', 'Last Name': 'Matthai', '$city': 'Ashburn', 'Geographical Preferences': None, 'company_visited': [14, 15, 100, 72, 149], 'Undergrad Concentration': None, 'First Name': 'Kristie'}, 'time': 1468526775045, 'distinct_id': '4878'}


In [39]:
people_email={}
for user_id, p in zip(np.where(people_roles=='Unknown')[0], people[np.where(people_roles=='Unknown')]):
    try:
        people_email[user_id]=p['properties']['Email']
    except KeyError:
        people_email[user_id]="Unknown"
id_unknown_email=[]
for user_id, email in people_email.items():
    if email=="Unknown":
        id_unknown_email.append(user_id)
id_unknown_email

[1539,
 1926,
 1826,
 1591,
 59,
 580,
 838,
 594,
 1885,
 741,
 1802,
 240,
 1136,
 1139,
 1913,
 380,
 1154,
 164,
 1190,
 951,
 1209,
 464,
 1253,
 723]

In [41]:
people[np.setdiff1d(np.where(people_roles=='Unknown')[0], np.array(id_unknown_email))]

array([ {'properties': {'Email': 'adam.halford@relishmba.com', '$timezone': 'America/New_York', 'Last Name': 'Halford', '$country_code': 'US', '$region': 'Virginia', '$city': 'Ashburn', 'First Name': 'Adam'}, 'distinct_id': '2980', 'time': 1453913906348},
       {'properties': {'Email': 'owen@lljcapital.com', '$timezone': 'America/New_York', 'Last Name': 'Larkin', '$country_code': 'US', '$region': 'Virginia', '$city': 'Ashburn', 'First Name': 'Owen'}, 'distinct_id': '3203', 'time': 1456434537847},
       {'properties': {'Email': 'diana.c.ma@relishmba.com', '$timezone': 'America/New_York', 'Last Name': 'Ma', '$country_code': 'US', '$region': 'Virginia', '$city': 'Ashburn', 'First Name': 'Diana'}, 'distinct_id': '3396', 'time': 1458857727518},
       {'properties': {'Email': 'kimkepchar@relishmba.com', '$timezone': 'America/New_York', 'Last Name': 'Kepchar', '$country_code': 'US', '$region': 'Virginia', '$city': 'Ashburn', 'First Name': 'Kim'}, 'distinct_id': '3185', 'time': 145615658503

In [ ]:
id_student=[]
for p in people:
    try:
        if p['properties']["Role"]=="student":
                id_student.append(p["distinct_id"])
    except KeyError:
        pass
id_student

In [46]:
user_id_length=[len(user_id) for user_id in id_student]

In [47]:
Counter(user_id_length)

Counter({2: 12, 3: 98, 4: 1727})

In [34]:
"Student".lower()

'student'